In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test = pd.read_csv('/kaggle/input/fake-news/test.csv')
train = pd.read_csv('/kaggle/input/fake-news/train.csv')
sub = pd.read_csv('/kaggle/input/fake-news/submit.csv')

In [ ]:
train.head(15)

In [ ]:
train['label'].value_counts()

In [ ]:
train = train.drop(['author', 'id'], axis = 1)

In [ ]:
#title
train.dropna(axis = 0,subset=['title'], inplace =True)

In [ ]:
train.reset_index(inplace=True)

In [ ]:
corpus = train['title']
corpus[:5]

In [ ]:
from keras.preprocessing.text import one_hot
from keras.layers import Embedding
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential

In [ ]:
voc_size = 15000

In [ ]:
onehotrepr = [ one_hot(words,voc_size) for words in corpus]

In [ ]:
onehotrepr

In [ ]:
sent_length = 50
embedded_docs = pad_sequences(onehotrepr, padding='pre', maxlen = sent_length)

In [ ]:
embedded_docs

In [ ]:
y = train['label']
y

In [ ]:
from keras.utils import to_categorical

In [ ]:
y =  to_categorical(y)
y

## SPLIT

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(embedded_docs, y, test_size = 0.2 , random_state=20)

In [ ]:
X_train.shape

## LSTM

In [ ]:
from keras.models import Sequential 
from keras.layers import LSTM, Dense, Dropout, Bidirectional


In [ ]:
model = Sequential()
model.add(Embedding(voc_size, 50, input_length=sent_length))
model.add(Bidirectional(LSTM(units = 1024 , return_sequences=True), input_shape = (50,50)))
model.add(Dropout(0.3))
model.add(Bidirectional(LSTM(units = 1024)))
model.add(Dropout(0.2))
model.add(Dense(units= 128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units= 2, activation='softmax'))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics =['accuracy'])

In [ ]:
hist = model.fit(X_train, y_train, validation_split=0.2, shuffle=True, batch_size=32, epochs=5)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
model.save('fake.h5')